In [ ]:
import pandas as pd

# Rutas de los archivo
ruta_archivo_contable = 'C:\\Users\\Documents\\Archivo_Contable.xlsx'
ruta_reporting = 'Z:\\Reporting\\2025\\Archivo_Reporting.xlsx'

# 1. Leer archivo descargado de contabilidad, saltando las 6 primeras filas ya que están en blanco
df_a3 = pd.read_excel(ruta_archivo_contable, sheet_name='Cuentas corrientes', header=6)

# 2. Vamos a eliminar los espacios en blanco al principio y al final de los nombres de las columnas porque a veces cuando cargas datos desde Excel, los nombres de las columnas pueden venir con espacios extra:
df_a3.columns = df_a3.columns.str.strip()

# 3. Eliminar columnas no deseadas si existen. En este caso, no necesitamos las columnas Punt., Documento, Saldo ni Contrapartida
columnas_a_eliminar = ["Punt.", "Documento", "Saldo", "Contrapartida"]
df_a3 = df_a3.drop(columns=[col for col in columnas_a_eliminar if col in df_a3.columns])

# 4. Eliminar filas que contienen "Total cuenta" en cualquier columna
df_a3 = df_a3[~df_a3.astype(str).apply(lambda row: row.str.contains("Total cuenta", na=False)).any(axis=1)]

# 5. Sobrescribir hoja "Reporting_mes" con los datos nuevos
with pd.ExcelWriter(ruta_reporting, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_a3.to_excel(writer, sheet_name='Reporting_mes', index=False)

# 6. Volver a leer la hoja actualizada
df = pd.read_excel(ruta_reporting, sheet_name='Reporting_mes')

# 7. Eliminar espacios en blanco en las columnas que puedan existir
df.columns = df.columns.str.strip()

# 8. Definir columnas clave
columnas_objetivo = ['Fecha', 'Concepto', 'Debe', 'Haber']
columnas_referencia = ['Cuenta', 'Descripción']

# Filtrar solo las columnas clave si están presentes (Opcional)
columnas_finales = [col for col in columnas_objetivo + columnas_referencia if col in df.columns]
df_filtrado = df[columnas_finales]

# 9. Guardar el archivo con la información actualiza
df_filtrado.to_excel("archivo_reporting_update.xlsx", index=False)